#### Tasks:	 
1)	Write a SQL Query combining vendors to calculate the Oct CPM by platform.

2)	Analyze the data from Vendor U and provide 3 insights and 3 recommendations.

3) 	Prepare a one slide executive summary of your findings from 1 & 2. 

#### Notes:
* CPM = Estimated revenue / Sold Impressions * 1000
* Vendors = vendor_s + vendor_u + vendor_t

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", -1)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

import numpy as np
import math

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("dark")
sns.set_context("paper", font_scale=1.8)
sns.set_palette("GnBu_d")

import pandas_profiling
import sqlite3

from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [2]:
!pwd

/Users/valmadrid/DataScienceBootcamp/Others


In [3]:
conn = sqlite3.connect("scmp.sqlite")
cur = conn.cursor()

In [4]:
cur.execute(
    """SELECT type, name, tbl_name, rootpage, sql FROM sqlite_master ORDER by type, tbl_name;"""
)
db = pd.DataFrame(cur.fetchall(), columns=[i[0] for i in cur.description])
db

,type,name,tbl_name,rootpage,sql
0,table,definitions_fields,definitions_fields,42,"CREATE TABLE ""definitions_fields"" (\n""Unnamed: 0"" TEXT,\n ""Standardized field names"" TEXT,\n ""Unnamed: 2"" TEXT,\n ""Unnamed: 3"" TEXT,\n ""Unnamed: 4"" TEXT,\n ""Unnamed: 5"" TEXT\n)"
1,table,definitions_groupings,definitions_groupings,41,"CREATE TABLE ""definitions_groupings"" (\n""Vendor "" TEXT,\n ""tagID"" TEXT,\n ""Zone Name"" TEXT,\n ""Platform"" TEXT,\n ""Ad Products"" TEXT\n)"
2,table,question,question,43,"CREATE TABLE ""question"" (\n""Time Limit: 3 hours"" TEXT,\n ""Unnamed: 1"" TEXT\n)"
3,table,vendor_b,vendor_b,24,"CREATE TABLE ""vendor_b"" (\n""DATE_INTERVAL"" TEXT,\n ""TAGS"" TEXT,\n ""SIZE"" TEXT,\n ""RECEIVED_IMPS"" INTEGER,\n ""IMPS"" INTEGER,\n ""REVENUE"" REAL,\n ""CLICKS"" INTEGER\n)"
4,table,vendor_s,vendor_s,2,"CREATE TABLE ""vendor_s"" (\n""TRAFFIC"" TEXT,\n ""TAG_ID"" INTEGER,\n ""TYPE"" TEXT,\n ""SITE"" TEXT,\n ""REQUESTS"" INTEGER,\n ""IMPRESSIONS"" INTEGER,\n ""FILL_RATE"" REAL,\n ""CPM"" REAL,\n ""EARNINGS"" REAL,\n ""UPDATED_DATE"" TIMESTAMP\n)"
5,table,vendor_t,vendor_t,25,"CREATE TABLE ""vendor_t"" (\n""WEBSITE"" TEXT,\n ""PLACEMENT"" TEXT,\n ""DAY"" TIMESTAMP,\n ""PUBLISHER_BILLABLE_VOLUME"" INTEGER,\n ""CLICK"" INTEGER,\n ""TEADS_BILLING_USD"" TEXT\n)"
6,table,vendor_u,vendor_u,11,"CREATE TABLE ""vendor_u"" (\n""PLATFORM"" TEXT,\n ""DATE"" TIMESTAMP,\n ""DEVICE_TYPE"" TEXT,\n ""VIDEO_IMPRESSIONS"" INTEGER,\n ""EARNINGS"" REAL,\n ""CLICKS"" INTEGER\n)"


In [5]:
scmp_excel = pd.read_excel("/Users/valmadrid/Downloads/SCMP.xlsx", sheet_name=None)

In [6]:
for name, table in scmp_excel.items():
    display(name)
    display(table.head())

'vendor_s'

,TRAFFIC,TAG_ID,TYPE,SITE,REQUESTS,IMPRESSIONS,FILL_RATE,CPM,EARNINGS,UPDATED_DATE
0,Combined,467649,DISPLAY,all traffic,511918,481718,0.94,0.14,67.75,2017-10-21
1,Combined,468681,DISPLAY,all traffic,439230,142757,0.33,0.15,20.80,2017-10-21
2,Combined,482946,DISPLAY,all traffic,167608,20837,0.12,0.13,2.74,2017-10-21
3,Combined,504560,DISPLAY,all traffic,175971,13414,0.08,0.27,3.66,2017-10-21
4,Combined,482944,DISPLAY,all traffic,284197,111739,0.39,0.13,14.15,2017-10-21


'vendor_u'

,PLATFORM,DATE,DEVICE_TYPE,VIDEO_IMPRESSIONS,EARNINGS,CLICKS
0,Unruly,2017-04-14,Desktop,4265,0.00,1073
1,Unruly,2017-02-13,Desktop,8585,0.00,1286
2,Unruly,2017-02-25,Desktop,2806,0.00,282
3,Unruly,2017-01-02,Desktop,1801,0.00,371
4,Unruly,2017-03-19,Desktop,6241,0.00,51


'vendor_b'

...

,Vendor,tagID,Zone Name,Platform,Ad Products
0,S,482944,ROS,Desktop,Island Ad 2
1,NaN,467645,ROS,Mobile Web,Island Ad 1
2,NaN,501047,ROS,Desktop,Island Ad 2
3,NaN,464637,ROS,Desktop,Super Leaderboard / Billboard
4,NaN,464635,ROS,Desktop,Island Ad 1


'Definitions - Fields'

,Unnamed: 0,Standardized field names,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Vendor,Date,Ad request,Sold Impressions,No Of Clicks,Estimated revenue
1,S,UPDATED_DATE,Requests,Impressions,NaN,Earnings
2,U*,DATE,NaN,video impressions,Clicks,Earnings
3,B,Date Interval,Received IMPS,IMPS,Clicks,Revenue
4,T,DAY,NaN,publisher_billable_volume,Click,teads_billing_usd


'Question'

,Time Limit: 3 hours,Unnamed: 1
0,NaN,NaN
1,1),Write a SQL Query combining vendors to calculate the Oct CPM by platform.
2,NaN,NaN
3,2),Analyze the data from Vendor U and provide 3 insights and 3 recommendations.
4,NaN,NaN


In [7]:
for table, df in scmp_excel.items():
    df.to_sql(table.lower().replace("-","_").replace(" ",""), conn, if_exists="replace", index=False)
    print("{} added to scmp.sqlite".format(table))

vendor_s added to scmp.sqlite
vendor_u added to scmp.sqlite
vendor_b added to scmp.sqlite
vendor_t added to scmp.sqlite
Definitions - Groupings added to scmp.sqlite
Definitions - Fields added to scmp.sqlite
Question added to scmp.sqlite


/Applications/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [8]:
cur.execute(
    """SELECT type, name, tbl_name, rootpage, sql FROM sqlite_master ORDER by type, tbl_name;"""
)
db = pd.DataFrame(cur.fetchall(), columns=[i[0] for i in cur.description])
db

,type,name,tbl_name,rootpage,sql
0,table,definitions_fields,definitions_fields,42,"CREATE TABLE ""definitions_fields"" (\n""Unnamed: 0"" TEXT,\n ""Standardized field names"" TEXT,\n ""Unnamed: 2"" TEXT,\n ""Unnamed: 3"" TEXT,\n ""Unnamed: 4"" TEXT,\n ""Unnamed: 5"" TEXT\n)"
1,table,definitions_groupings,definitions_groupings,41,"CREATE TABLE ""definitions_groupings"" (\n""Vendor "" TEXT,\n ""tagID"" TEXT,\n ""Zone Name"" TEXT,\n ""Platform"" TEXT,\n ""Ad Products"" TEXT\n)"
2,table,question,question,43,"CREATE TABLE ""question"" (\n""Time Limit: 3 hours"" TEXT,\n ""Unnamed: 1"" TEXT\n)"
3,table,vendor_b,vendor_b,24,"CREATE TABLE ""vendor_b"" (\n""DATE_INTERVAL"" TEXT,\n ""TAGS"" TEXT,\n ""SIZE"" TEXT,\n ""RECEIVED_IMPS"" INTEGER,\n ""IMPS"" INTEGER,\n ""REVENUE"" REAL,\n ""CLICKS"" INTEGER\n)"
4,table,vendor_s,vendor_s,2,"CREATE TABLE ""vendor_s"" (\n""TRAFFIC"" TEXT,\n ""TAG_ID"" INTEGER,\n ""TYPE"" TEXT,\n ""SITE"" TEXT,\n ""REQUESTS"" INTEGER,\n ""IMPRESSIONS"" INTEGER,\n ""FILL_RATE"" REAL,\n ""CPM"" REAL,\n ""EARNINGS"" REAL,\n ""UPDATED_DATE"" TIMESTAMP\n)"
5,table,vendor_t,vendor_t,25,"CREATE TABLE ""vendor_t"" (\n""WEBSITE"" TEXT,\n ""PLACEMENT"" TEXT,\n ""DAY"" TIMESTAMP,\n ""PUBLISHER_BILLABLE_VOLUME"" INTEGER,\n ""CLICK"" INTEGER,\n ""TEADS_BILLING_USD"" TEXT\n)"
6,table,vendor_u,vendor_u,11,"CREATE TABLE ""vendor_u"" (\n""PLATFORM"" TEXT,\n ""DATE"" TIMESTAMP,\n ""DEVICE_TYPE"" TEXT,\n ""VIDEO_IMPRESSIONS"" INTEGER,\n ""EARNINGS"" REAL,\n ""CLICKS"" INTEGER\n)"


In [9]:
cur.execute("""SELECT * FROM vendor_s LIMIT 5;""")
s = pd.DataFrame(cur.fetchall(), columns = [i[0] for i in cur.description])
display(s.head())

cur.execute("""SELECT * FROM vendor_t LIMIT 5;""")
t = pd.DataFrame(cur.fetchall(), columns = [i[0] for i in cur.description])
display(t.head())

cur.execute("""SELECT * FROM vendor_u LIMIT 5;""")
u = pd.DataFrame(cur.fetchall(), columns = [i[0] for i in cur.description])
display(u.head())

,TRAFFIC,TAG_ID,TYPE,SITE,REQUESTS,IMPRESSIONS,FILL_RATE,CPM,EARNINGS,UPDATED_DATE
0,Combined,467649,DISPLAY,all traffic,511918,481718,0.94,0.14,67.75,2017-10-21 00:00:00
1,Combined,468681,DISPLAY,all traffic,439230,142757,0.33,0.15,20.80,2017-10-21 00:00:00
2,Combined,482946,DISPLAY,all traffic,167608,20837,0.12,0.13,2.74,2017-10-21 00:00:00
3,Combined,504560,DISPLAY,all traffic,175971,13414,0.08,0.27,3.66,2017-10-21 00:00:00
4,Combined,482944,DISPLAY,all traffic,284197,111739,0.39,0.13,14.15,2017-10-21 00:00:00


,WEBSITE,PLACEMENT,DAY,PUBLISHER_BILLABLE_VOLUME,CLICK,TEADS_BILLING_USD
0,32277 - scmp.com,48540 - MS-inRead-Mobile-scmp.com,2017-10-08 00:00:00,904,250,40.38 USD
1,32277 - scmp.com,48540 - MS-inRead-Mobile-scmp.com,2017-10-08 00:00:00,4382,969,32.25 USD
2,32277 - scmp.com,48540 - MS-inRead-Mobile-scmp.com,2017-10-08 00:00:00,868,234,39.27 USD
3,32277 - scmp.com,48540 - MS-inRead-Mobile-scmp.com,2017-10-08 00:00:00,3738,237,9.26 USD
4,32277 - scmp.com,48540 - MS-inRead-Mobile-scmp.com,2017-10-08 00:00:00,3270,1078,48.09 USD


,PLATFORM,DATE,DEVICE_TYPE,VIDEO_IMPRESSIONS,EARNINGS,CLICKS
0,Unruly,2017-04-14 00:00:00,Desktop,4265,0.00,1073
1,Unruly,2017-02-13 00:00:00,Desktop,8585,0.00,1286
2,Unruly,2017-02-25 00:00:00,Desktop,2806,0.00,282
3,Unruly,2017-01-02 00:00:00,Desktop,1801,0.00,371
4,Unruly,2017-03-19 00:00:00,Desktop,6241,0.00,51


In [11]:
cur.execute("""SELECT * FROM definitions_groupings;""")
mapping = pd.DataFrame(cur.fetchall(),
                     columns=[i[0] for i in cur.description])
mapping

,Vendor,tagID,Zone Name,Platform,Ad Products
0,S,482944,ROS,Desktop,Island Ad 2
1,None,467645,ROS,Mobile Web,Island Ad 1
2,None,501047,ROS,Desktop,Island Ad 2
3,None,464637,ROS,Desktop,Super Leaderboard / Billboard
4,None,464635,ROS,Desktop,Island Ad 1
5,None,467644,ROS,Desktop,Island Ad 1
6,None,490370,ROS,Desktop,Island Ad 2
7,None,463809,homepage,Desktop,Island Ad 1
8,None,482939,ROS,Desktop,Island Ad 1
9,None,482946,ROS,Desktop,Island Ad 2


#### Vendor S

In [10]:
cur.execute("""SELECT MAX(UPDATED_DATE) FROM vendor_s;""")
display(cur.fetchall())
cur.execute("""SELECT MIN(UPDATED_DATE) FROM vendor_s;""")
display(cur.fetchall())

[('2017-11-04 00:00:00',)]

[('2017-10-12 00:00:00',)]

In [15]:
cur.execute(
    """
    SELECT 
        s.UPDATED_Date as date,
        s.TAG_ID as tag_id,
        map."Zone Name" as zone_name,
        map.Platform as platform, 
        map."Ad Products" as ad_products, 
        s.IMPRESSIONS as sold_impressions, 
        s.EARNINGS as est_revenue, 
        s.CPM as cpm
    FROM vendor_s as s
    LEFT JOIN definitions_groupings as map
        ON s.TAG_ID = map.tagID
    WHERE UPDATED_DATE BETWEEN '2017-10-01' AND '2017-10-31';
    """
)
vendor_s = pd.DataFrame(cur.fetchall(),
                     columns=[i[0] for i in cur.description])
vendor_s

,date,tag_id,zone_name,platform,ad_products,sold_impressions,est_revenue,cpm
0,2017-10-21 00:00:00,467649,ROS,Desktop,Super Leaderboard / Billboard,481718,67.75,0.14
1,2017-10-21 00:00:00,468681,ROS,Desktop,Super Leaderboard / Billboard,142757,20.80,0.15
2,2017-10-21 00:00:00,482946,ROS,Desktop,Island Ad 2,20837,2.74,0.13
3,2017-10-21 00:00:00,504560,ROS,Mobile Web,Island Ad 2,13414,3.66,0.27
4,2017-10-21 00:00:00,482944,ROS,Desktop,Island Ad 2,111739,14.15,0.13
...,...,...,...,...,...,...,...,...
244,2017-10-12 00:00:00,504560,ROS,Mobile Web,Island Ad 2,1440,0.04,0.03
245,2017-10-12 00:00:00,463812,ROS,Mobile Web,Island Ad 1,11867,3.43,0.29
246,2017-10-12 00:00:00,463809,homepage,Desktop,Island Ad 1,18579,2.04,0.11
247,2017-10-12 00:00:00,496179,homepage,Mobile Web,Island Ad 2,52939,28.33,0.54


In [48]:
cur.execute("""DROP TABLE IF EXISTS oct_table;""")

cur.execute("""
    CREATE TABLE oct_table AS
        SELECT 
            s.UPDATED_Date as date,
            s.TAG_ID as map,
            map."Zone Name" as zone_name,
            map.Platform as platform, 
            map."Ad Products" as ad_products, 
            s.IMPRESSIONS as sold_impressions, 
            s.EARNINGS as est_revenue, 
            s.CPM as cpm
        FROM vendor_s as s
        LEFT JOIN definitions_groupings as map
            ON s.TAG_ID = map.tagID
        WHERE UPDATED_DATE BETWEEN '2017-10-01' AND '2017-10-31'
        UNION
        SELECT 
            u.DATE as date,
            u.DEVICE_TYPE as map,
            map."Zone Name" as zone_name,
            map.Platform as platform, 
            map."Ad Products" as ad_products, 
            u.VIDEO_IMPRESSIONS as sold_impressions, 
            u.EARNINGS as est_revenue,
            u.EARNINGS as cpm
        FROM vendor_u as u
        LEFT JOIN definitions_groupings as map
            ON lower(u.DEVICE_TYPE) = lower(map.tagID)
        WHERE DATE BETWEEN '2017-10-01' AND '2017-10-31';
    """)

cur.execute(
    """SELECT type, name, tbl_name, rootpage, sql FROM sqlite_master ORDER by type, tbl_name;"""
)
db = pd.DataFrame(cur.fetchall(), columns=[i[0] for i in cur.description])
db

,type,name,tbl_name,rootpage,sql
0,table,definitions_fields,definitions_fields,42,"CREATE TABLE ""definitions_fields"" (\n""Unnamed: 0"" TEXT,\n ""Standardized field names"" TEXT,\n ""Unnamed: 2"" TEXT,\n ""Unnamed: 3"" TEXT,\n ""Unnamed: 4"" TEXT,\n ""Unnamed: 5"" TEXT\n)"
1,table,definitions_groupings,definitions_groupings,41,"CREATE TABLE ""definitions_groupings"" (\n""Vendor "" TEXT,\n ""tagID"" TEXT,\n ""Zone Name"" TEXT,\n ""Platform"" TEXT,\n ""Ad Products"" TEXT\n)"
2,table,oct_table,oct_table,44,"CREATE TABLE oct_table(\n date NUM,\n map INT,\n zone_name TEXT,\n platform TEXT,\n ad_products TEXT,\n sold_impressions INT,\n est_revenue REAL,\n cpm REAL\n)"
3,table,question,question,43,"CREATE TABLE ""question"" (\n""Time Limit: 3 hours"" TEXT,\n ""Unnamed: 1"" TEXT\n)"
4,table,vendor_b,vendor_b,24,"CREATE TABLE ""vendor_b"" (\n""DATE_INTERVAL"" TEXT,\n ""TAGS"" TEXT,\n ""SIZE"" TEXT,\n ""RECEIVED_IMPS"" INTEGER,\n ""IMPS"" INTEGER,\n ""REVENUE"" REAL,\n ""CLICKS"" INTEGER\n)"
5,table,vendor_s,vendor_s,2,"CREATE TABLE ""vendor_s"" (\n""TRAFFIC"" TEXT,\n ""TAG_ID"" INTEGER,\n ""TYPE"" TEXT,\n ""SITE"" TEXT,\n ""REQUESTS"" INTEGER,\n ""IMPRESSIONS"" INTEGER,\n ""FILL_RATE"" REAL,\n ""CPM"" REAL,\n ""EARNINGS"" REAL,\n ""UPDATED_DATE"" TIMESTAMP\n)"
6,table,vendor_t,vendor_t,25,"CREATE TABLE ""vendor_t"" (\n""WEBSITE"" TEXT,\n ""PLACEMENT"" TEXT,\n ""DAY"" TIMESTAMP,\n ""PUBLISHER_BILLABLE_VOLUME"" INTEGER,\n ""CLICK"" INTEGER,\n ""TEADS_BILLING_USD"" TEXT\n)"
7,table,vendor_u,vendor_u,11,"CREATE TABLE ""vendor_u"" (\n""PLATFORM"" TEXT,\n ""DATE"" TIMESTAMP,\n ""DEVICE_TYPE"" TEXT,\n ""VIDEO_IMPRESSIONS"" INTEGER,\n ""EARNINGS"" REAL,\n ""CLICKS"" INTEGER\n)"


#### Vendor U

In [13]:
cur.execute("""SELECT MAX(DATE) FROM vendor_u;""")
display(cur.fetchall())
cur.execute("""SELECT MIN(DATE) FROM vendor_u;""")
display(cur.fetchall())

[('2017-10-29 00:00:00',)]

[('2017-01-01 00:00:00',)]

In [128]:
cur.execute("""
    SELECT 
        u.DATE as date,
        u.DEVICE_TYPE as device_type,
        groupings."Zone Name" as zone_name,
        groupings.Platform as platform, 
        groupings."Ad Products" as ad_products,
        u.VIDEO_IMPRESSIONS as sold_impressions,
        (CAST(u.VIDEO_IMPRESSIONS as REAL) / map.total_sold_impressions * map.total_est_revenue)*1.00 as est_revenue,
        (CAST(u.VIDEO_IMPRESSIONS as REAL) / map.total_sold_impressions * map.total_est_revenue)/CAST(u.VIDEO_IMPRESSIONS as REAL)*1000.00 as cpm
    FROM vendor_u as u
    LEFT JOIN (SELECT
                    DATE as date,
                    CAST(sum(VIDEO_IMPRESSIONS) as REAL) as total_sold_impressions,
                    CAST(sum(EARNINGS) as REAL) as total_est_revenue
                FROM vendor_u
                WHERE DATE BETWEEN '2017-10-01' AND '2017-10-31'
                GROUP by DATE) as map
        USING (DATE)
    LEFT JOIN definitions_groupings as groupings
        ON lower(u.DEVICE_TYPE) = lower(groupings.tagID)
    WHERE
        (DATE BETWEEN '2017-10-01' AND '2017-10-31') &
        (DEVICE_TYPE NOT LIKE "%UNKNOWN%")
    GROUP by DATE, DEVICE_TYPE;
    """)

vendor_u = pd.DataFrame(cur.fetchall(),
                             columns=[i[0] for i in cur.description])
vendor_u

,date,device_type,zone_name,platform,ad_products,sold_impressions,est_revenue,cpm
0,2017-10-01 00:00:00,Desktop,ROS,Desktop,Full Page Ad/Inread Video,7476,23.96,3.20
1,2017-10-01 00:00:00,Mobile & Tablet,ROS,Mobile Web,Full Page Ad/Inread Video,896,2.87,3.20
2,2017-10-02 00:00:00,Desktop,ROS,Desktop,Full Page Ad/Inread Video,6714,23.14,3.45
3,2017-10-02 00:00:00,Mobile & Tablet,ROS,Mobile Web,Full Page Ad/Inread Video,19600,67.57,3.45
4,2017-10-03 00:00:00,Desktop,ROS,Desktop,Full Page Ad/Inread Video,6747,12.32,1.83
5,2017-10-03 00:00:00,Mobile & Tablet,ROS,Mobile Web,Full Page Ad/Inread Video,14800,27.02,1.83
6,2017-10-04 00:00:00,Desktop,ROS,Desktop,Full Page Ad/Inread Video,9248,26.52,2.87
7,2017-10-04 00:00:00,Mobile & Tablet,ROS,Mobile Web,Full Page Ad/Inread Video,22003,63.11,2.87
8,2017-10-05 00:00:00,Desktop,ROS,Desktop,Full Page Ad/Inread Video,1965,10.09,5.13
9,2017-10-05 00:00:00,Mobile & Tablet,ROS,Mobile Web,Full Page Ad/Inread Video,16992,87.23,5.13


In [131]:
cur.execute("""DROP TABLE IF EXISTS oct_table;""")

cur.execute("""
    CREATE TABLE oct_table AS
        SELECT 
            s.UPDATED_Date as date,
            s.TAG_ID as map,
            map."Zone Name" as zone_name,
            map.Platform as platform, 
            map."Ad Products" as ad_products, 
            s.IMPRESSIONS as sold_impressions, 
            s.EARNINGS as est_revenue, 
            s.CPM as cpm
        FROM vendor_s as s
        LEFT JOIN definitions_groupings as map
            ON s.TAG_ID = map.tagID
        WHERE UPDATED_DATE BETWEEN '2017-10-01' AND '2017-10-31'
        UNION
            SELECT 
                u.DATE as date,
                u.DEVICE_TYPE as device_type,
                groupings."Zone Name" as zone_name,
                groupings.Platform as platform, 
                groupings."Ad Products" as ad_products,
                u.VIDEO_IMPRESSIONS as sold_impressions,
                (CAST(u.VIDEO_IMPRESSIONS as REAL) / map2.total_sold_impressions * map2.total_est_revenue)*1.00 as est_revenue,
                (CAST(u.VIDEO_IMPRESSIONS as REAL) / map2.total_sold_impressions * map2.total_est_revenue)/CAST(u.VIDEO_IMPRESSIONS as REAL)*1000.00 as cpm
            FROM vendor_u as u
            LEFT JOIN (SELECT
                            DATE as date,
                            CAST(sum(VIDEO_IMPRESSIONS) as REAL) as total_sold_impressions,
                            CAST(sum(EARNINGS) as REAL) as total_est_revenue
                        FROM vendor_u
                        WHERE DATE BETWEEN '2017-10-01' AND '2017-10-31'
                        GROUP by DATE) as map2
            USING (DATE)
    LEFT JOIN definitions_groupings as groupings
        ON lower(u.DEVICE_TYPE) = lower(groupings.tagID)
    WHERE
        (DATE BETWEEN '2017-10-01' AND '2017-10-31') &
        (DEVICE_TYPE NOT LIKE "%UNKNOWN%")
    GROUP by DATE, DEVICE_TYPE;
    """)

cur.execute(
    """SELECT * FROM oct_table;"""
)

oct_cpm = pd.DataFrame(cur.fetchall(), columns=[i[0] for i in cur.description])
oct_cpm

,date,map,zone_name,platform,ad_products,sold_impressions,est_revenue,cpm
0,2017-10-01 00:00:00,Desktop,ROS,Desktop,Full Page Ad/Inread Video,7476,23.96,3.20
1,2017-10-01 00:00:00,Mobile & Tablet,ROS,Mobile Web,Full Page Ad/Inread Video,896,2.87,3.20
2,2017-10-02 00:00:00,Desktop,ROS,Desktop,Full Page Ad/Inread Video,6714,23.14,3.45
3,2017-10-02 00:00:00,Mobile & Tablet,ROS,Mobile Web,Full Page Ad/Inread Video,19600,67.57,3.45
4,2017-10-03 00:00:00,Desktop,ROS,Desktop,Full Page Ad/Inread Video,6747,12.32,1.83
...,...,...,...,...,...,...,...,...
302,2017-10-30 00:00:00,482944,ROS,Desktop,Island Ad 2,199316,9.48,0.05
303,2017-10-30 00:00:00,482946,ROS,Desktop,Island Ad 2,26607,28.02,1.05
304,2017-10-30 00:00:00,496179,homepage,Mobile Web,Island Ad 2,8223,2.95,0.36
305,2017-10-30 00:00:00,501047,ROS,Desktop,Island Ad 2,123448,5.97,0.05


#### Vendor T

In [132]:
cur.execute("""SELECT MAX(DAY) FROM vendor_t;""")
display(cur.fetchall())
cur.execute("""SELECT MIN(DAY) FROM vendor_t;""")
display(cur.fetchall())

[('2017-11-03 00:00:00',)]

[('2017-10-08 00:00:00',)]

In [ ]:
cur.execute("""SELECT * FROM vendor_t WHERE DAY BETWEEN '2017-10-01' AND '2017-10-31';""")
vendor_t = pd.DataFrame(cur.fetchall(), columns = [i[0] for i in cur.description])
display(vendor_t)
display(vendor_t.info())

In [ ]:
vendor_t.PLACEMENT.value_counts()

In [ ]:
cur.execute("""SELECT * FROM definition_groupings WHERE tagID LIKE "%SS%" OR tagID LIKE "%MS%" OR tagID LIKE "%CP%";""")
t_map = pd.DataFrame(cur.fetchall(), columns = [i[0] for i in cur.description])
t_map.tagID.iloc[3] = "79052 - MS-inRead-Desktop-scmp.com/magazines/style"
t_map

In [ ]:
q = """
SELECT 
    t_map."Zone Name" as Zone_Name,
    t_map.Platform, 
    t_map."Ad Products" as Ad_Products,
    t.PUBLISHER_BILLABLE_VOLUME as Sold_Impressions, 
    t.TEADS_BILLING_USD as Est_Revenue
FROM vendor_t as t
LEFT JOIN t_map as t_map
ON t.PLACEMENT = t_map.tagID;
"""
vendor_t_final = pysqldf(q)
vendor_t_final["Est_Revenue"] = vendor_t_final.Est_Revenue.map(lambda x: float(str(x).strip(" USD")))
vendor_t_final["CPM"] = vendor_t_final["Est_Revenue"]/vendor_t_final["Sold_Impressions"]*1000
vendor_t_final["Vendor"] = "T"
vendor_t_final

#### Final Vendor Table

In [ ]:
q = """
SELECT *
FROM vendor_s_final
UNION
SELECT *
FROM vendor_u_final
UNION
SELECT *
FROM vendor_t_final
;
"""
vendor_final = pysqldf(q)
vendor_final

#### Total October 2017 CPM is:

In [ ]:
sum(vendor_final.CPM)

In [ ]:
temp = vendor_final.groupby(["Vendor"]).sum().reset_index()[["Vendor", "CPM"]]
plt.figure(figsize=(8, 7))
graph = plt.bar(temp.Vendor, temp.CPM, color=["C0", "C2", "C4"])
plt.title("October 2017 CPM: HK${}".format(round(sum(vendor_final.CPM), 2)), fontsize=20)
plt.xlabel("Vendor")
plt.yticks(range(0, 11000, 1000))
plt.ylabel("CPM")
for bar, x in zip(graph, temp.CPM.values):
    plt.text(bar.get_x() + bar.get_width() / 2.0,
             bar.get_height() + 200,
             "HK${} ({}%)".format(round(x, 2),
                               round(x / sum(vendor_final.CPM) * 100, 2)),
             fontsize=15,
             ha="center",
             va="bottom")
plt.show()

#### Vendor U Analysis

In [ ]:
cur.execute("""SELECT * FROM vendor_u;""")
u = pd.DataFrame(cur.fetchall(), columns = [i[0] for i in cur.description])
display(u.info())
for d in u.DATE.unique():
    
    i = u[(u.DATE == d) & (u.DEVICE_TYPE == "Desktop")].index[0]
    u.at[i, "CPM"] = u[
        (u.DATE == d)
        & (u.DEVICE_TYPE == "Desktop")].VIDEO_IMPRESSIONS.values[0] / (
            u[(u.DATE == d)
              & (u.DEVICE_TYPE == "Desktop")].VIDEO_IMPRESSIONS.values[0] +
            u[(u.DATE == d)
              &
              (u.DEVICE_TYPE == "Mobile & Tablet")].VIDEO_IMPRESSIONS.values[0]
        )
    u["EARNINGS"].iloc[i] = u.CPM.iloc[i] / 1000 * u.VIDEO_IMPRESSIONS.iloc[i]
    
    i = u[(u.DATE == d) & (u.DEVICE_TYPE == "Mobile & Tablet")].index[0]
    u.at[i, "CPM"] = u[
        (u.DATE == d)
        & (u.DEVICE_TYPE == "Mobile & Tablet")].VIDEO_IMPRESSIONS.values[0] / (
            u[(u.DATE == d)
              & (u.DEVICE_TYPE == "Desktop")].VIDEO_IMPRESSIONS.values[0] +
            u[(u.DATE == d)
              &
              (u.DEVICE_TYPE == "Mobile & Tablet")].VIDEO_IMPRESSIONS.values[0]
        )
    u["EARNINGS"].iloc[i] = u.CPM.iloc[i] / 1000 * u.VIDEO_IMPRESSIONS.iloc[i]
    
u.info()

In [ ]:
q = """
SELECT 
    u_map."Zone Name" as Zone_Name,
    u_map.Platform, 
    u_map."Ad Products" as Ad_Products, 
    u.DATE as Date,
    u.VIDEO_IMPRESSIONS as Sold_Impressions, 
    u.CLICKS as Clicks,
    u.EARNINGS as Est_Revenue,
    u.CPM
FROM u as u
LEFT JOIN u_map as u_map
ON u.DEVICE_TYPE = u_map.tagID
WHERE DEVICE_TYPE != "[UNKNOWN]";
"""
df_u = pysqldf(q)
display(df_u.shape)
df_u

In [ ]:
df_u.Date = pd.to_datetime(df_u["Date"], errors="ignore")
df_u["Month"] = df_u.Date.map(lambda x: x.month_name())
df_u["M"] = df_u.Date.map(lambda x: x.month)
df_u["Year"] = df_u.Date.map(lambda x: x.year)
df_u.head()

In [ ]:
df_u[["Platform", "CPM"]].groupby("Platform").sum().plot(kind="barh",
                                                         color="C4",
                                                         figsize=(10, 6))
plt.legend("")
plt.title("Vendor U CPM per Platform")
plt.show()

In [ ]:
temp = df_u[["M", "Month", "CPM"]].groupby(["M", "Month"]).sum().reset_index()
temp.set_index(temp.Month, inplace=True, drop=True)
temp[["Month", "CPM"]].plot(kind="barh",
                            stacked=True,
                            color="C4",
                            figsize=(10, 6))
plt.legend("")
plt.yticks()
plt.title("Vendor U CPM per Month")
plt.show()

In [ ]:
sum(sheet1.EARNINGS)

In [ ]:
sum(df_u.Est_Revenue)